# Debug

In [ ]:
# ===== Cell: Deep Dive Diagnostic for Sample NCBI525 =====
# CodeGuardian: This cell is a high-precision diagnostic tool to uncover why
# sample NCBI525 has missing data. We will trace the data from its source
# to its final state, pinpointing the exact point of failure.

import sys
from pathlib import Path
import pandas as pd
import numpy as np
import anndata
import logging
import os
import rootutils

# --- 1. 配置 (与您的环境保持一致) ---
# Use rootutils to find the project root based on a marker file (e.g., .git)
try:
    PROJECT_ROOT = rootutils.find_root(search_from=__file__, indicator=".git")
except NameError:
    # Fallback for interactive environments like Jupyter
    PROJECT_ROOT = rootutils.find_root(search_from=".", indicator=".git")

if not PROJECT_ROOT.exists():
    raise FileNotFoundError(f"项目根目录不存在: {PROJECT_ROOT}")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# a. 原始 HEST 数据集目录 (用于加载 .h5ad)
RAW_DATA_DIR = PROJECT_ROOT / "data" / "hest_1k_original"
# b. d1 notebook 生成的已处理数据目录
# 动态查找包含 nodes.parquet 的目录，避免硬编码路径
def find_processed_data_dir(base_path: Path) -> Path | None:
    for p in base_path.iterdir():
        if p.is_dir() and (p / "nodes.parquet").exists():
            return p
    return None

PROCESSED_DATA_DIR = find_processed_data_dir(PROJECT_ROOT / "data")
if PROCESSED_DATA_DIR is None:
    raise FileNotFoundError(f"在 '{PROJECT_ROOT / 'data'}' 目录下未找到任何包含 'nodes.parquet' 的已处理数据文件夹。")

# c. 目标样本
TARGET_SAMPLE_ID = "NCBI525"

logger = logging.getLogger("ncbi525_diagnostic")
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
    logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

# --- 2. 量化问题：对比 AnnData 与 Parquet 中的 Spot 数量 ---
logger.info(f"--- 1. 量化问题: 对比 '{TARGET_SAMPLE_ID}' 的 Spot 数量 ---")
adata_path = RAW_DATA_DIR / "st" / f"{TARGET_SAMPLE_ID}.h5ad"
if not adata_path.exists():
    raise FileNotFoundError(f"未找到 AnnData 文件: {adata_path}")

nodes_path = PROCESSED_DATA_DIR / "nodes.parquet"
if not nodes_path.exists():
    raise FileNotFoundError(f"未找到处理后的节点文件: {nodes_path}")

try:
    adata = anndata.read_h5ad(adata_path)
except Exception as exc:
    logger.error("❌ 在读取 AnnData 文件时发生错误", exc_info=True)
    raise

adata_spot_ids = set(adata.obs.index)
num_adata_spots = len(adata_spot_ids)
logger.info(f"原始 AnnData (.h5ad) 中 '{TARGET_SAMPLE_ID}' 的 Spot 总数: {num_adata_spots}")

try:
    nodes_df = pd.read_parquet(nodes_path)
except Exception:
    logger.error("❌ 在读取 nodes.parquet 文件时发生错误", exc_info=True)
    raise

processed_nodes_for_sample = nodes_df[nodes_df["sample_id"] == TARGET_SAMPLE_ID].copy()
if processed_nodes_for_sample.empty:
    logger.warning("⚠️ 在处理数据集中未找到目标样本。后续检查将被跳过。")
else:
    num_processed_spots = len(processed_nodes_for_sample)
    logger.info(f"已处理 Parquet 中 '{TARGET_SAMPLE_ID}' 的 Spot 数量: {num_processed_spots}")

    discrepancy = num_adata_spots - num_processed_spots
    loss_percentage = (discrepancy / num_adata_spots) * 100 if num_adata_spots > 0 else 0.0
    logger.info(f"数量差异: {discrepancy} 个 spots 被丢弃 (损失率: {loss_percentage:.2f}%)")

    if discrepancy == 0:
        logger.info("✅ 数量一致，问题可能不在于数据丢失。")
    else:
        logger.warning("🔥 数量严重不匹配！这是问题的核心。")

# --- 3. 定位“失踪”与“幸存”的 Spot ---
if not processed_nodes_for_sample.empty:
    logger.info("\n--- 2. 定位失踪与幸存的 Spot ---")

    def parse_spot_id_from_path_v2(path_str: str | Path) -> str | None:
        try:
            return Path(path_str).stem
        except Exception:
            return None

    valid_image_paths = processed_nodes_for_sample["image_path"].dropna().tolist()
    processed_spot_ids_from_path = {
        spot_id
        for spot_id in (parse_spot_id_from_path_v2(path) for path in valid_image_paths)
        if spot_id
    }

    missing_spot_ids = adata_spot_ids - processed_spot_ids_from_path
    logger.info(f"找到了 {len(missing_spot_ids)} 个 '失踪' 的 Spot。示例: {list(missing_spot_ids)[:5]}")

    # --- 4. 审问“失踪者”：验证它们对应的瓦片文件是否有效 ---
    logger.info("\n--- 3. 根本原因分析: 检查 '失踪' Spot 对应的瓦片文件 ---")
    
    # 从幸存的 spot 推断瓦片目录和文件扩展名
    tile_dir = None
    tile_ext = None
    if valid_image_paths:
        first_path = Path(valid_image_paths[0])
        tile_dir = first_path.parent
        tile_ext = first_path.suffix
        logger.info(f"从幸存的 spot 推断出瓦片目录: {tile_dir}")
        logger.info(f"从幸存的 spot 推断出文件扩展名: {tile_ext}")

    if not tile_dir or not tile_ext:
        logger.warning("⚠️ 无法从 'nodes.parquet' 推断瓦片目录或扩展名，跳过文件检查。")
    else:
        logger.info(f"对 {min(5, len(missing_spot_ids))} 个 '失踪' Spot 的文件进行检查...")
        num_missing_files = 0
        num_empty_files = 0

        for spot_id in list(missing_spot_ids)[:5]:
            # 基于推断的目录和扩展名构建瓦片文件的完整路径
            path_to_check = tile_dir / f"{spot_id}{tile_ext}"
            
            if not path_to_check.is_absolute():
                path_to_check = (PROJECT_ROOT / path_to_check).resolve()
            
            log_msg = f"  - Spot '{spot_id}', 路径: {path_to_check}"
            if not path_to_check.exists():
                log_msg += " -> ❌ 文件不存在"
                num_missing_files += 1
            elif path_to_check.stat().st_size == 0:
                log_msg += " -> ❌ 文件大小为 0"
                num_empty_files += 1
            else:
                log_msg += f" -> ✅ 文件存在且大小为 {path_to_check.stat().st_size} bytes"
            logger.info(log_msg)

        logger.info("\n--- 诊断结论 ---")
        if num_missing_files > 0:
            logger.error("🔥 主要问题: 许多瓦片文件从未被生成或路径错误，导致它们在预处理的第一步就被过滤掉了。")
        if num_empty_files > 0:
            logger.error("🔥 主要问题: 许多瓦片文件被生成为空文件 (0KB)，同样导致它们被过滤。")
        if num_missing_files == 0 and num_empty_files == 0:
            logger.info("✅ '失踪' spot 对应的瓦片文件似乎存在且有效。问题可能在预处理的筛选逻辑中，而不是文件丢失。")

        if num_missing_files > 0 or num_empty_files > 0:
            print("\n" + "=" * 80)
            print("💡 行动建议:")
            print("1. 请回到您最初的切片脚本 (可能在 `d1_...ipynb` 之前或之中)。")
            print(f"2. 检查处理样本 '{TARGET_SAMPLE_ID}' 时是否有任何错误日志。")
            print("3. 可能的原因包括：")
            print("   - WSI 文件本身存在问题 (例如，部分区域损坏)。")
            print("   - 写入瓦片文件时遇到了磁盘空间不足或权限问题。")
            print("   - 脚本中的坐标转换逻辑对于这个特定样本可能存在 bug。")
            print(f"4. 解决方案: 需要重新运行对 `{TARGET_SAMPLE_ID}` 样本的瓦片切割过程，并密切监控日志以确保所有瓦片都被正确生成。")
            print("=" * 80)


2025-11-06 10:06:09,850 - INFO - --- 1. 量化问题: 对比 'NCBI525' 的 Spot 数量 ---
2025-11-06 10:06:09,927 - INFO - 原始 AnnData (.h5ad) 中 'NCBI525' 的 Spot 总数: 777
2025-11-06 10:06:10,323 - INFO - 已处理 Parquet 中 'NCBI525' 的 Spot 数量: 112
2025-11-06 10:06:10,324 - INFO - 数量差异: 665 个 spots 被丢弃 (损失率: 85.59%)
2025-11-06 10:06:10,325 - WARNING - 🔥 数量严重不匹配！这是问题的核心。
2025-11-06 10:06:10,326 - INFO - 
--- 2. 定位失踪与幸存的 Spot ---
2025-11-06 10:06:10,327 - INFO - 找到了 777 个 '失踪' 的 Spot。示例: ['GTGTACCTTGGCTACG-1', 'GTGCTCAAGTACTGTC-1', 'GTGATTCGCCGCTCAA-1', 'CACCTAATCAGTTTAC-1', 'AGCACTTAAGGACGCC-1']
2025-11-06 10:06:10,327 - INFO - 
--- 3. 根本原因分析: 检查 '失踪' Spot 对应的瓦片文件 ---
2025-11-06 10:06:10,328 - INFO - 从幸存的 spot 推断出瓦片目录: /cpfs01/projects-HDD/cfff-afe2df89e32e_HDD/jjh_19301050235/my_data/hest_data/hest_output/NCBI525_tiles
2025-11-06 10:06:10,328 - INFO - 从幸存的 spot 推断出文件扩展名: .png
2025-11-06 10:06:10,328 - INFO - 对 5 个 '失踪' Spot 的文件进行检查...
2025-11-06 10:06:10,334 - INFO -   - Spot 'GTGTACCTTGGCTACG-1', 路径: /cpfs01/


💡 行动建议:
1. 请回到您最初的切片脚本 (可能在 `d1_...ipynb` 之前或之中)。
2. 检查处理样本 'NCBI525' 时是否有任何错误日志。
3. 可能的原因包括：
   - WSI 文件本身存在问题 (例如，部分区域损坏)。
   - 写入瓦片文件时遇到了磁盘空间不足或权限问题。
   - 脚本中的坐标转换逻辑对于这个特定样本可能存在 bug。
4. 解决方案: 需要重新运行对 `NCBI525` 样本的瓦片切割过程，并密切监控日志以确保所有瓦片都被正确生成。


## 随机抽取十个样本进行监测

In [ ]:
# ===== Cell: Widespread Data Loss Diagnostic (10 Random Samples) =====
# CodeGuardian: This cell automates the diagnostic process from the previous cell
# for a random subset of samples to determine if the data loss issue is systemic.

import sys
from pathlib import Path
import pandas as pd
import numpy as np
import anndata
import logging
import os
import random
import rootutils
from tqdm.auto import tqdm

# --- 1. 配置 (与您的环境保持一致) ---
# Use rootutils to find the project root based on a marker file (e.g., .project-root)
try:
    PROJECT_ROOT = rootutils.find_root(search_from=__file__, indicator=".project-root")
except NameError:
    # Fallback for interactive environments like Jupyter
    PROJECT_ROOT = rootutils.find_root(search_from=".", indicator=".project-root")

if not PROJECT_ROOT.exists():
    raise FileNotFoundError(f"项目根目录不存在: {PROJECT_ROOT}")

# a. 原始 HEST 数据集目录 (用于加载 .h5ad)
RAW_DATA_DIR = PROJECT_ROOT / "data" / "hest_1k_original"
# b. d1 notebook 生成的已处理数据目录
# 动态查找包含 nodes.parquet 的目录，避免硬编码路径
def find_processed_data_dir(base_path: Path) -> Path | None:
    # 假设已处理数据在 hest_hugo_6nei_correct_parquet_data/train 目录下
    candidate_path = base_path / "hest_hugo_6nei_correct_parquet_data" / "train"
    if candidate_path.exists() and (candidate_path / "nodes.parquet").exists():
        return candidate_path
    
    # 如果找不到，再进行通用搜索
    for p in base_path.iterdir():
        if p.is_dir() and (p / "nodes.parquet").exists():
            return p
    return None

PROCESSED_DATA_DIR = find_processed_data_dir(PROJECT_ROOT / "data")
if PROCESSED_DATA_DIR is None:
    raise FileNotFoundError(f"在 '{PROJECT_ROOT / 'data'}' 目录下未找到任何包含 'nodes.parquet' 的已处理数据文件夹。")

# --- 2. 封装诊断逻辑为一个函数 ---
logger = logging.getLogger("widespread_diagnostic")
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
    logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

def diagnose_sample_data_loss(sample_id: str, raw_data_dir: Path, processed_nodes_df: pd.DataFrame) -> dict:
    """
    Performs a deep-dive diagnostic for a single sample to check for data loss.
    """
    result = {
        "sample_id": sample_id,
        "adata_spots": 0,
        "processed_spots": 0,
        "loss_percentage": 0.0,
        "diagnosis": "N/A",
        "details": ""
    }
    
    # 1. 加载 AnnData
    adata_path = raw_data_dir / "st" / f"{sample_id}.h5ad"
    if not adata_path.exists():
        result["diagnosis"] = "Error"
        result["details"] = f"AnnData file not found at {adata_path}"
        logger.error(f"[{sample_id}] ❌ {result['details']}")
        return result
    
    try:
        adata = anndata.read_h5ad(adata_path)
        adata_spot_ids = set(adata.obs.index)
        result["adata_spots"] = len(adata_spot_ids)
    except Exception as e:
        result["diagnosis"] = "Error"
        result["details"] = f"Failed to read AnnData file: {e}"
        logger.error(f"[{sample_id}] ❌ {result['details']}")
        return result

    # 2. 从已处理数据中筛选
    sample_nodes_df = processed_nodes_df[processed_nodes_df["sample_id"] == sample_id]
    result["processed_spots"] = len(sample_nodes_df)

    if result["adata_spots"] == 0:
        result["diagnosis"] = "Warning"
        result["details"] = "Source AnnData has 0 spots."
        logger.warning(f"[{sample_id}] ⚠️ {result['details']}")
        return result
        
    # 3. 计算差异
    discrepancy = result["adata_spots"] - result["processed_spots"]
    result["loss_percentage"] = (discrepancy / result["adata_spots"]) * 100
    
    if discrepancy == 0:
        result["diagnosis"] = "OK"
        result["details"] = "All spots accounted for."
        logger.info(f"[{sample_id}] ✅ {result['details']} ({result['adata_spots']} spots)")
        return result

    logger.warning(
        f"[{sample_id}] 🔥 Mismatch found! "
        f"AnnData: {result['adata_spots']}, Parquet: {result['processed_spots']}. "
        f"Loss: {discrepancy} spots ({result['loss_percentage']:.2f}%)"
    )

    # 4. 根本原因分析
    valid_image_paths = sample_nodes_df["image_path"].dropna().tolist()
    if not valid_image_paths:
        result["diagnosis"] = "Critical"
        result["details"] = "No valid image paths found in processed data, but spots exist in AnnData."
        logger.error(f"[{sample_id}] ❌ {result['details']}")
        return result
    
    # 从路径中解析出 spot_id
    processed_spot_ids_from_path = {Path(p).stem for p in valid_image_paths}
    missing_spot_ids = adata_spot_ids - processed_spot_ids_from_path

    # 推断瓦片目录和扩展名
    first_path = Path(valid_image_paths[0])
    tile_dir, tile_ext = first_path.parent, first_path.suffix

    # 抽查几个丢失的 spot
    num_missing_files = 0
    for spot_id in list(missing_spot_ids)[:5]: # Check up to 5 missing spots
        path_to_check = tile_dir / f"{spot_id}{tile_ext}"
        if not path_to_check.exists():
            num_missing_files += 1
            
    if num_missing_files > 0:
        result["diagnosis"] = "File Not Found"
        result["details"] = f"At least {num_missing_files} of 5 checked 'missing' spots correspond to non-existent tile image files."
        logger.error(f"[{sample_id}] ❌ {result['details']}")
    else:
        result["diagnosis"] = "Logic/Filter Issue"
        result["details"] = "'Missing' spots seem to have valid image files. The issue is likely in the data filtering logic."
        logger.warning(f"[{sample_id}] ⚠️ {result['details']}")

    return result

# --- 3. 主执行逻辑 ---
logger.info("--- Starting Widespread Data Loss Diagnostic ---")

# a. 加载完整的节点数据
nodes_path = PROCESSED_DATA_DIR / "nodes.parquet"
if not nodes_path.exists():
    raise FileNotFoundError(f"未找到处理后的节点文件: {nodes_path}")
full_nodes_df = pd.read_parquet(nodes_path)

# b. 获取所有样本ID并随机抽样
all_sample_ids = full_nodes_df['sample_id'].unique().tolist()
num_samples_to_check = min(10, len(all_sample_ids)) # Check up to 10 samples
selected_samples = random.sample(all_sample_ids, num_samples_to_check)

logger.info(f"Randomly selected {len(selected_samples)} samples for diagnosis: {selected_samples}")

# c. 循环诊断
diagnostic_results = []
for sample_id in tqdm(selected_samples, desc="Diagnosing Samples"):
    diagnostic_results.append(
        diagnose_sample_data_loss(sample_id, RAW_DATA_DIR, full_nodes_df)
    )

# d. 汇总并展示结果
summary_df = pd.DataFrame(diagnostic_results)
summary_df['loss_percentage'] = summary_df['loss_percentage'].map('{:.2f}%'.format)

# 调整列顺序以便查看
display_cols = ['sample_id', 'diagnosis', 'adata_spots', 'processed_spots', 'loss_percentage', 'details']
summary_df = summary_df[display_cols]

print("\n" + "="*80)
print("                       Widespread Data Loss Diagnostic Summary")
print("="*80)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 60)
print(summary_df)
print("="*80)

# e. 最终结论
critical_issues = summary_df[summary_df['diagnosis'].isin(['File Not Found', 'Critical'])]
if not critical_issues.empty:
    logger.error(f"\n🔥 CRITICAL FINDING: {len(critical_issues)} out of {num_samples_to_check} checked samples have significant data loss, likely due to missing tile files.")
    logger.error("This indicates a systemic problem in the initial tile generation/saving process. Please review the upstream scripts.")
else:
    logger.info("\n✅ All randomly checked samples appear to be healthy or have minor issues. The problem might be isolated to specific samples like NCBI525.")

2025-11-06 10:33:19,847 - INFO - --- Starting Widespread Data Loss Diagnostic ---
2025-11-06 10:33:21,528 - INFO - Randomly selected 10 samples for diagnosis: ['NCBI574', 'INT20', 'NCBI876', 'TENX115', 'NCBI510', 'MEND51', 'MEND151', 'NCBI783', 'MISC52', 'INT12']


Diagnosing Samples:   0%|          | 0/10 [00:00<?, ?it/s]

2025-11-06 10:33:21,909 - INFO - [NCBI574] ✅ All spots accounted for. (1505 spots)
2025-11-06 10:33:22,276 - WARNING - [INT20] 🔥 Mismatch found! AnnData: 4860, Parquet: 4805. Loss: 55 spots (1.13%)
2025-11-06 10:33:22,299 - ERROR - [INT20] ❌ At least 5 of 5 checked 'missing' spots correspond to non-existent tile image files.
2025-11-06 10:33:22,387 - WARNING - [NCBI876] 🔥 Mismatch found! AnnData: 1064, Parquet: 348. Loss: 716 spots (67.29%)
2025-11-06 10:33:22,390 - ERROR - [NCBI876] ❌ At least 5 of 5 checked 'missing' spots correspond to non-existent tile image files.
2025-11-06 10:33:22,490 - WARNING - [TENX115] 🔥 Mismatch found! AnnData: 3886, Parquet: 1403. Loss: 2483 spots (63.90%)
2025-11-06 10:33:22,498 - ERROR - [TENX115] ❌ At least 5 of 5 checked 'missing' spots correspond to non-existent tile image files.
2025-11-06 10:33:22,607 - WARNING - [NCBI510] 🔥 Mismatch found! AnnData: 525, Parquet: 167. Loss: 358 spots (68.19%)
2025-11-06 10:33:22,609 - ERROR - [NCBI510] ❌ At least 5


                       Widespread Data Loss Diagnostic Summary
  sample_id       diagnosis  adata_spots  processed_spots loss_percentage  \
0   NCBI574              OK         1505             1505           0.00%   
1     INT20  File Not Found         4860             4805           1.13%   
2   NCBI876  File Not Found         1064              348          67.29%   
3   TENX115  File Not Found         3886             1403          63.90%   
4   NCBI510  File Not Found          525              167          68.19%   
5    MEND51              OK          416              416           0.00%   
6   MEND151  File Not Found         2600             2583           0.65%   
7   NCBI783  File Not Found         3869             2254          41.74%   
8    MISC52  File Not Found          229              228           0.44%   
9     INT12              OK         1451             1451           0.00%   

                                                       details  
0                      

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os

# Check whether all the paths in the nodesdf exist
logger.info("--- 全面检查 `nodes.parquet` 中的所有文件路径 ---")

# 确保 full_nodes_df 已从前一个单元格加载
if 'full_nodes_df' not in locals():
    logger.error("❌ `full_nodes_df` 未定义。请先运行上面的单元格。")
else:
    # 获取所有唯一的、非空的图像路径
    image_paths_to_check = full_nodes_df['image_path'].dropna().unique()
    total_paths = len(image_paths_to_check)
    logger.info(f"将要检查 {total_paths} 个唯一的文件路径...")

    def check_path_exists(path_str):
        """检查单个文件路径是否存在，如果不存在则返回路径字符串。"""
        absolute_path = PROJECT_ROOT / path_str
        if not absolute_path.exists():
            return path_str
        return None

    missing_paths = []
    # 使用 ThreadPoolExecutor 并行检查文件
    with ThreadPoolExecutor(max_workers=os.cpu_count() or 4) as executor:
        # 使用 tqdm 显示进度条
        results = list(tqdm(executor.map(check_path_exists, image_paths_to_check), total=total_paths, desc="正在验证文件路径"))
    
    # 从结果中过滤出所有非 None 的项（即缺失的路径）
    missing_paths = [path for path in results if path is not None]

    # --- 报告结果 ---
    print("\n" + "="*80)
    print("                       文件路径完整性检查报告")
    print("="*80)
    if not missing_paths:
        logger.info(f"✅ 成功！所有 {total_paths} 个文件路径均有效且文件存在。")
    else:
        num_missing = len(missing_paths)
        logger.error(f"🔥 失败！在 {total_paths} 个路径中发现 {num_missing} 个缺失文件。")
        logger.error("这表明在数据处理的某个阶段，瓦片文件丢失或路径记录错误。")
        
        # 打印一些缺失文件的例子
        print("\n缺失文件示例 (最多显示10个):")
        for p in missing_paths[:10]:
            print(f"  - {p}")
        if num_missing > 10:
            print(f"  ... 以及其他 {num_missing - 10} 个文件。")
    print("="*80)

2025-11-06 10:39:03,025 - INFO - --- 全面检查 `nodes.parquet` 中的所有文件路径 ---
2025-11-06 10:39:03,477 - INFO - 将要检查 919173 个唯一的文件路径...


正在验证文件路径:   0%|          | 0/919173 [00:00<?, ?it/s]

2025-11-06 10:39:55,525 - INFO - ✅ 成功！所有 919173 个文件路径均有效且文件存在。



                       文件路径完整性检查报告
